# Comparação de Filmes pelas Reviews no IMDB

## Processamento de Linguagem Natural

Maira Zabuscha de Lima

21008214

Professor Jesús P. Mena-Chalco

O presente trabalho propõe um algoritmo de comparação de filmes baseado no processamento dos textos das reviews deixadas pelos usuários no site IMDB [(www.imdb.com)](https://www.imdb.com/). Para isso, foi criado um programa Python que extrai os textos das reviews para gerar um banco de dados de treinamento de alguns filmes classificados manualmente. Então, um filme ainda não visto e não classificado passará pelo algoritmo para que sejam exibidos os filmes de treinamento mais semelhantes a este. São exibidas também as notas para comparação.

### Bibliotecas

In [1]:
import re
import csv
import requests
import numpy
from scipy.spatial import distance

### Entrada de usuário

Preencha o código e nome do filme a ser comparado.

In [2]:
movie = 'tt4786282'
name = 'Lights Out'

### Exprexssões Regulares

Aqui são definidas duas expressões regulares:
* tag - Localiza o texto das reviews dentro de sua tag HTML.
* regex - Localiza todas as palavras dentro de cada review.

In [3]:
tag = re.compile(r'<div class="text show-more__control">(.+?)</div>', re.DOTALL)

regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ]+"

### Stopwords

O arquivo stopwords.txt contém uma lista de palavras em inglês para serem ignoradas na leitura das reviews.

In [4]:
Stopwords = set([]) 
sw = open("stopwords.txt",'r')
for s in sw.readlines():
    Stopwords.add(s.strip().lower())
sw.close()
print("{0} stopwords".format(len(Stopwords)))

379 stopwords


### Filmes classificados

O arquivo ratings.csv pode ser obtido em sua página de usuário do IMDB em uma url do tipo:

https://www.imdb.com/user/ur24136776/ratings

In [5]:
f = open('ratings-short.csv', 'r')
reader = csv.reader(f)
lines = list(reader)
f.close()
lines = lines[1:]
print("{0} filmes".format(len(lines)))
print(lines[14])

53 filmes
['tt1703199', '10', '2011-09-08', 'Grave Encounters', 'https://www.imdb.com/title/tt1703199/', 'movie', '6.1', '92', '2011', 'Horror', '45492', '2011-04-22', 'The Vicious Brothers, Colin Minihan, Stuart Ortiz']


### Variáveis

As seguintes variáveis vão guardar as informações que o algoritmo utilizará:
* Document - dicionário com elementos do tipo ('código do filme', list()) onde a lista contém as palavras retiradas das reviews exceto as stopwords.
* Title - dicionário com elementos do tipo ('código do filme', 'título do filme').
* Rating - dicionário com elementos do tipo ('código do filme', 'rating do filme').
* Vocabulary - conjunto com todas as palavras do vocabulário de todos os filmes.

In [6]:
Document = dict([])
Title = dict([])
Rating = dict([])
Vocabulary = set([])

### Função leFilme

Essa função acessa o site imdb.com e retira as reviews de todos os filmes listados em ratings.csv, preenchendo as variáveis acima.

In [7]:
def leFilme(movieCode, movieTitle, movieRating):
    url = 'https://www.imdb.com/title/' + movieCode + '/reviews'
    #print(url)
    res = requests.get(url)
    words = list()
    if res.status_code == requests.codes.ok:
        conteudo = res.text
        reviews = tag.findall(conteudo)
        if len(reviews) > 0:
            txt = '\n'.join(reviews).replace('&#39;', '\'').replace('<br/>', ' ').replace('&quot;', '\"').replace('&amp;', '&')
            txt = re.sub(r'[^\x00-\x7f]',r'', txt)
            words = re.findall(regex, txt)
    if len(words) > 0:
        Document[movieCode] = list()
        for w in words:
            if w not in Stopwords and len(w)>=3:
                Document[movieCode].append(w.lower())
        if len(Document[movieCode]) > 0:
            Vocabulary.update(Document[movieCode])
            Title[movieCode] = movieTitle
            Rating[movieCode] = movieRating
        else:
            del Document[movieCode]

### Leitura das reviews

Aqui a função acima é chamada para cada filme, inclusive o filme provido pelo usuário.

In [8]:
for line in lines:
    leFilme(line[0], line[3], line[1])
leFilme(movie, name, '0')

D = len(Document)
V = len(Vocabulary)
print("{} documentos".format(D))
print("{} palavras".format(V))

54 documentos
20261 palavras


### Matriz de freqüências

A matriz de freqüências M é uma matriz com V linhas, para cada palavra, e D colunas, para cada filme. Cada coluna é o vetor de um filme, com cada elemento do vetor sendo a freqüência de uma palavra do vocabulário nesse filme.

Essa etapa pode ser bem demorada.

In [9]:
M = numpy.zeros((V, D))
documents  = list(Document.keys())
vocabulary = list(Vocabulary)
for j in range(0, D):
    d = documents[j]       
    for i in range(0, V):
        w = vocabulary[i]       
        M[i,j] = float(Document[d].count(w))
print(len(M), len(M[0]))

20261 54


### Matriz TF-IDF

TF (Term-Frequency) é o vetor do documento normalizado pelo seu tamanho.

IDF (Inverse Document Frequency) é um vetor cujo tamanho é o tamanho do vocabulário e pra cada palavra do vocabulário calcula:

idf(w) = log(D / D(w)), onde D(w) é a quantidade de documentos que contém w.

Quanto mais comum for a palavra, mais seu valor de IDF tenderá a zero.

A matriz TF-IDF é obtida, então, multiplicando-se cada vetor TF pelo vetor IDF e concatenado-os.

In [10]:
idf = numpy.count_nonzero(M, axis=1)
idf = [D / x for x in idf]
idf = numpy.log(idf)
Sd = numpy.sum(M, axis=0)
TFIDF = numpy.copy(M)
for j in range(0, D): 
    TFIDF[:,j] = [x / Sd[j] for x in TFIDF[:,j]]
    TFIDF[:,j] = numpy.multiply(TFIDF[:,j], idf)

### Medidas de distância

Aqui definimos dois dicionários para administrar todas as medidas de distância disponíveis para, então, avaliarmos a diferença que a escolha da medida causa no resultado.

#### chebyshev
\begin{aligned}
max_i = |u_i - v_i|
\end{aligned}

#### euclidean
\begin{aligned}
\sqrt{\sum{(u_i - v_i)^2}}
\end{aligned}

#### cosine
\begin{aligned}
1 - \frac{u \cdot v}{||u||_2 ||v||_2}
\end{aligned}

In [11]:
dist_f = {'euclidean': distance.euclidean, 
          'chebyshev': distance.chebyshev, 
          'cosine': distance.cosine}
dist = {'euclidean': numpy.ones(D-1) * numpy.nan, 
          'chebyshev': numpy.ones(D-1) * numpy.nan, 
          'cosine': numpy.ones(D-1) * numpy.nan,
          'tf-idf': numpy.ones(D-1) * numpy.nan}

### Cálculo da distância

O cálculo da distância é repetido para todas as medidas acima na matriz M e também para a distância cosseno na matriz TF-IDF.

In [12]:
for i in range(0, D-1): 
    for df in dist_f.keys():
        dist[df][i] = dist_f[df](M[:,i], M[:,-1])
    
    dist['tf-idf'][i] = distance.cosine(TFIDF[:,i], TFIDF[:,-1])

### Cálculo da similaridade

A similaridade é calculada como (1 - distancia normalizada).

In [13]:
similarity = dict()
for df in dist.keys():
    similarity[df] = 1 - (dist[df] - numpy.nanmin(dist[df])) / (numpy.nanmax(dist[df]) - numpy.nanmin(dist[df]))

### Resultados

Os 5 filmes mais similares por cada uma das medidas é exbido, além de sua nota extraída de ratings.csv. Em parênteses, o valor da similaridade.

In [14]:
for df in dist.keys():
    idx = numpy.argsort(similarity[df])
    txt = '\nTitulos mais similares a ' + name + ' por distancia ' + df + ' e suas notas:'
    for i in range(-1, -6, -1):
        txt += '\n{1} - {0} ({2:.2f})'.format(Title[documents[idx[i]]], Rating[documents[idx[i]]].rjust(2), similarity[df][idx[i]])
    
    print(txt)


Titulos mais similares a Lights Out por distancia euclidean e suas notas:
 6 - The Ritual (1.00)
 8 - The Void (0.91)
10 - Grave Encounters (0.88)
 5 - As Above, So Below (0.87)
 6 - The Tunnel (0.87)

Titulos mais similares a Lights Out por distancia chebyshev e suas notas:
 6 - The Tunnel (1.00)
 8 - The Void (0.99)
 5 - As Above, So Below (0.99)
10 - The Skeleton Key (0.97)
 5 - Gok-seong (0.97)

Titulos mais similares a Lights Out por distancia cosine e suas notas:
 6 - The Ritual (1.00)
 8 - The Cabin in the Woods (0.92)
 6 - It Follows (0.90)
 5 - Chernobyl Diaries (0.89)
 5 - Gok-seong (0.89)

Titulos mais similares a Lights Out por distancia tf-idf e suas notas:
10 - Grave Encounters (1.00)
 6 - It Follows (0.92)
 5 - Paranormal Activity 3 (0.75)
 6 - The Ritual (0.53)
 4 - Ouija: Origin of Evil (0.47)


### Conclusão

Há margem para interpretação do resultado obtido por esse algoritmo pois utiliza textos escritos por usuários do site que contém tanto expressão de sentimento em relação à qualidade do filme, quanto termos descritivos do enredo do filme. Por esse motivo, os filmes são muito mais fortemente agrupados por gênero do que por qualidade, de modo que, esse não é um algoritmo que pode ser considerado de recomendação. Fica a cargo do usuário interpretar os resultados, já que são exibidas também as notas dos filmes semelhantes.

Outro ponto a ser observado é que os resultados ficam restritos aos filmes classificados pelo usuário, de modo que se o usuário classifica irregularmente os diferentes gêneros, os resultados podem ser inesperados e mesmo inúteis caso o usuário selecione para teste um filme de um gênero que geralmente não classifica.